### Importing Dependencies

In [5]:
import requests
import os
import psycopg2
import time
import threading
import pandas as pd
from finvizfinance.quote import finvizfinance
from io import StringIO
from concurrent.futures import ThreadPoolExecutor

### Scraping Stock Stickers on S&P 500 Index

In [2]:
url = 'https://www.slickcharts.com/sp500'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)

html_content = StringIO(response.text)
tables = pd.read_html(html_content)

sp500_table = tables[0]
tickers = list(sp500_table['Symbol'])
tickers = [ticker.replace('.', '-') for ticker in tickers]

### Fetching headlines of the stocks from FinViz 

In [4]:
news_dfs = {}

def get_news(ticker):
    try:
        print(f"Fetching news for {ticker}...\n")
        stock = finvizfinance(ticker)
        news_df = stock.ticker_news()
        return news_df
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")
        return None

with ThreadPoolExecutor(max_workers=8) as executor:
    for i, ticker in enumerate(tickers):
        news_dfs[ticker] = executor.submit(get_news, ticker)
        if (i + 1) % 20 == 0:
            time.sleep(0.001)

Fetching news for MSFT...Fetching news for AAPL...

Fetching news for NVDA...
Fetching news for AMZN...
Fetching news for META...
Fetching news for GOOGL...
Fetching news for BRK-B...
Fetching news for GOOG...
Fetching news for LLY...
Fetching news for AVGO...
Fetching news for TSLA...
Fetching news for JPM...
Fetching news for UNH...
Fetching news for V...
Fetching news for XOM...
Fetching news for MA...
Fetching news for JNJ...
Fetching news for PG...
Fetching news for HD...
Fetching news for COST...
Fetching news for MRK...
Fetching news for ABBV...
Fetching news for AMD...
Fetching news for CRM...
Fetching news for CVX...
Fetching news for ADBE...
Fetching news for NFLX...
Fetching news for WMT...
Fetching news for BAC...
Fetching news for ACN...
Fetching news for KO...
Fetching news for PEP...
Fetching news for TMO...
Fetching news for MCD...
Fetching news for LIN...
Fetching news for DIS...
Fetching news for CSCO...
Fetching news for ABT...
Fetching news for WFC...
Fetching news 

### Connecting to Postgres Database